# LightSSS：轻量级仿真快照

在仿真过程中，LightSSS 会使用 fork() 函数记录进程的快照。当检测到 bug 时，它会被唤醒，并从 bug 发生前最新的快照开始生成若干个周期的波形。

使用 LightSSS 转储波形。

In [1]:
%%bash

cd ../ && source env.sh >/dev/null

cd $NOOP_HOME

./build/emu -i $AM_HOME/apps/hello/build/hello-riscv64-xs.bin \
--diff $NEMU_HOME/build/riscv64-nemu-interpreter-so \
--enable-fork \
2 > lightsss.err

如果您看到“the oldest checkpoint start to dump wave and dump nemu log...”，则表示 LightSSS 正在运行。之后，模拟将从现有的最新快照重新开始。